In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import sys,os,re
import sympy as sp
import math
from itertools import product

In [3]:
from Groups import GroupOhD, NUM_ELEMENTS
from Functions import extract_repr, rec_dd
from Init import init_S, init_R, init_G

In [44]:
# check Nsgm
from pandas import read_pickle

coef_Npi=read_pickle('coeffsNpi.pkl')
coef_Nsgm=read_pickle('coeffsNsgm.pkl')

g='2Oh'; irrep='G1u'; l='l1'; mom=(0,0,0); occ='N0\\pi0'; occ2='a'

for momNpi,val in coef_Npi[g][irrep][l][mom][occ][occ2].items():
    print(momNpi,val['cg5;g5'])
print()
for momNpi,val in coef_Nsgm[g]['G1g'][l][mom]['N0\\sigma0'][occ2].items():
    print(momNpi,val['cg5;1'])

((0, 0, 0), (0, 0, 0)) [1.+0.j 0.+0.j 0.+0.j 0.+0.j]

((0, 0, 0), (0, 0, 0)) [1.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [9]:
#initialize group
group_2Oh = GroupOhD()

#spin1212 matrix
S = extract_repr( group_2Oh, init_S )

#spin1 matrix
Rot3d = extract_repr( group_2Oh, init_R )

#irrep G1g
Gamma = extract_repr( group_2Oh, init_G['2Oh']['G1g'] )

# Check 1: Elements in $O_h^D$

A map between the convention used in Basak paper (hep-lat/0506029) and the numeration from Bernard paper (0806.4495) can be found in NUM_ELEMENTS for all the proper rotations.

In [3]:
### Read F output #
###################

def read_showline(line):
    if re.match("(\w+)_(\w+) <- list\(\)",line):
        return None
    head = r"r\w*_\w+\[\[\s*(\d+)\]\] <- array\(dim = c\((\d+)\s*,\s*(\d+)\)\)"
    normalline = r"r\w*_\w+\[\[\s*(\d+)\]\]\[(\d+),(\d+)\] <-\s+(.+) \+\s+(.+)\*1\.i"

    obj_m = re.match(head,line)
    if obj_m:
        key, dim_r, dim_c = obj_m.groups()
        return key, dim_r, dim_c# 
    else:
        obj_m = re.match(normalline,line)
        if obj_m:
            key, dim_r, dim_c, n_re, n_im = obj_m.groups()
            return key, dim_r, dim_c, n_re, n_im
        else:
            print(line,'not recognized')
            return None

In [4]:
## read matrix from Ferenc output
file_matrix='little_group_projector_0.show'

R_F = {}
S_F = {}
G_F = {}
for lab in ['R','IR']:
    #select only rotations (not IR)
    !grep rp_$lab $file_matrix >> tmp_R.show
    !grep rspin0_$lab $file_matrix >> tmp_S.show
    !grep rtarget_$lab $file_matrix >> tmp_G.show
    
    for _file,_dict in zip(['tmp_R.show','tmp_S.show','tmp_G.show'],[R_F,S_F,G_F]):
        _dict[lab] = {}
        

        #read lines
        with open(_file,'r') as reader:
            for iline,line in enumerate(reader.readlines()):
                out = read_showline(line)
                if out is None:
                    print('skip line ',iline)
                    continue
                elif len(out)==3:
                    k, dr, dc = out
                    assert(k not in _dict[lab]),k+' line %d: '%(iline)+str(line)
                    _dict[lab][k] = np.empty((int(dr),int(dc)),dtype=np.complex)
                else:
                    k, i, j, vre, vim = out
                    _dict[lab][k][int(i)-1,int(j)-1] = float(vre) +1j*float(vim)
        #delete tmp file
        !rm $_file
        #convert to sympy matrices
        _dict[lab] = { k:(sp.Matrix(x)).applyfunc(sp.nsimplify) for k,x in _dict[lab].items()}

skip line  0
skip line  0
skip line  0
skip line  0
skip line  0
skip line  0


In [12]:
#check R 3x3 (spin 1)
for R_name, R_number in NUM_ELEMENTS.items(): #elements of double-cover of Oh without inversion
    #R
    assert(Rot3d[R_name]==R_F['R'][str(R_number)]),R_name+' differs'
    
    #IR
    IR_name = 'I_s'+R_name
    assert(Rot3d[IR_name]==R_F['IR'][str(R_number)]),IR_name+' differs'

In [13]:
#check S 4x4 (spin1212)
for R_name, R_number in NUM_ELEMENTS.items(): #elements of double-cover of Oh without inversion
    #R
    assert(S[R_name]==S_F['R'][str(R_number)]),R_name+' differs'
    
    #IR
    IR_name = 'I_s'+R_name
    assert((S[IR_name]).applyfunc(sp.nsimplify)==S_F['IR'][str(R_number)]),IR_name+' differs'

In [14]:
#check G 2x2 (G1g irrep)
for R_name, R_number in NUM_ELEMENTS.items(): #elements of double-cover of Oh without inversion
    #R
    assert((Gamma[R_name]).applyfunc(sp.nsimplify)==G_F['R'][str(R_number)]),R_name+' differs'
    
    #IR
    IR_name = 'I_s'+R_name
    assert((Gamma[IR_name]).applyfunc(sp.nsimplify)==G_F['IR'][str(R_number)]),IR_name+' differs'

# Check 2: check Delta coeff with Marcus out

In [17]:
def readoutMarcus(line):
    header="# \[\w+\] lg\s+(\w+)\sirrep\s+(\w+) mu\s+(\d+) beta\s+(\d+)"
    normalline="annihilation (.+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(.+)\s+(.+)"
    obj_m = re.match(header,line)
    if obj_m:
        lg, irrep, mu, beta = obj_m.groups()
        return lg, irrep, int(mu), int(beta), None
    else:
        obj_m = re.match(normalline,line)
        if obj_m:
            _type, ig, _is, jg, js, vre, vim = obj_m.groups()
            vre=0 if abs(float(vre))<1e-12 else float(vre)
            vim=0 if abs(float(vim))<1e-12 else float(vim)
            return _type, int(ig)*4+int(_is), int(jg)*4+int(js), vre+1j*vim
        else:
            return None

In [18]:
Marcus_p = rec_dd()

for iline,line in enumerate(open('out','r').readlines()):
    content = readoutMarcus(line)
    if content is None:
        continue
    elif len(content)==5:
        lg, irrep, mu, beta, _ = content
#         print(iline,'--',mu,beta)
        Marcus_p[lg][irrep]['%d%d'%(mu,beta)]['P'] = sp.zeros(12,dtype=complex) 
        Marcus_p[lg][irrep]['%d%d'%(mu,beta)]['P_ONB'] = sp.zeros(12,dtype=complex) 
        Marcus_p[lg][irrep]['%d%d'%(mu,beta)]['R_ONB'] = sp.zeros(12,dtype=complex) 
        writer = Marcus_p[lg][irrep]['%d%d'%(mu,beta)] 
    elif len(content)==4:
#         print('data')
        _type, i, j, val = content
        _type = '_'.join(_type.split())
        writer[_type][i,j] = val
Marcus_p = Marcus_p.todict()